In [1]:
import numpy as np
from os import walk
import re
import requests
from bs4 import BeautifulSoup,Comment
import re
import pandas as pd

In [2]:
import pandas as pd
import os
import shutil

In [3]:
FOLDER_PATH = './dataset/foodnetwork/'
# test_file = '0a5ae5d3ccf0e0012ccb68716c9af39c.html'

In [4]:
def normalize_string(string):
    return re.sub(
        r'\s+', ' ',
        string.replace(
            '\xa0', ' ').replace(  # &nbsp;
            '\n', ' ').replace(
            '\t', ' ').strip()
    )

In [5]:
TIME_REGEX = re.compile(
    r'(\D*(?P<hours>\d+)\s*(hours|hrs|hr|h|Hours|H))?(\D*(?P<minutes>\d+)\s*(minutes|mins|min|m|Minutes|M))?')

def get_minutes(element):
    try:
        tstring = element.get_text()
        if '-' in tstring:
            tstring = tstring.split('-')[1]  # sometimes formats are like this: '12-15 minutes'
        matched = TIME_REGEX.search(tstring)

        minutes = int(matched.groupdict().get('minutes') or 0)
        minutes += 60 * int(matched.groupdict().get('hours') or 0)

        return minutes
    except AttributeError:  # if dom_element not found or no matched
        return 0

In [6]:
done_list = []

In [7]:
null_list, rating_review_list, times_list, ingredient_list, direction_list = ([] for i in range(5))
count=0

In [8]:
# for root, _, files in walk(FOLDER_PATH):
#     for name in files:
#         if name in done_list:
#             print('The file has been scrapped.')
#             continue
#         soup_i = BeautifulSoup(open(FOLDER_PATH + name, 'rb'), 'html.parser')
#         if soup_i.findAll('a'):    #make sure there is no blank HTML
#             if ('http://www.foodnetwork.com/recipes/' in soup_i.text and soup_i.find("meta", property="og:title")): # make sure the page is for recipe
#                 title = soup_i.find("meta", property="og:title")['content']
#                 if soup_i.find('div',class_="rm-block lead hreview-aggregate review"):
#                     rating_tmp = re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)
#                 else:
#                     rating_tmp = []

#                 if soup_i.findAll('span', {'class':"lgbtn-text"}):
#                     review_tmp = re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)
#                 else:
#                     review_tmp = []

#                 rating_review_list.append({'Title': title, 
#                                            'Rating': float(re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)[0])\
#                                                      if rating_tmp else np.nan,
#                                            'Reviews': int(re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)[0])\
#                                                       if review_tmp else np.nan
#                                           })
#                 times_list.append({'Title': title, 
#                                    'Prep Time (min)': get_minutes(soup_i.find('dd',class_="prepTime clrfix"))\
#                                                       if soup_i.find('dd',class_="prepTime clrfix") else np.nan,
#                                    'Cook Time (min)': get_minutes(soup_i.find('dd',class_="cookTime clrfix"))\
#                                                       if soup_i.find('dd',class_="cookTime clrfix") else np.nan,
#                                    'Total Time (min)': get_minutes(soup_i.find('dd',class_="head duration clrfix"))\
#                                                      if soup_i.find('dd',class_="head duration clrfix") else np.nan
#                                   })
#                 ingredients = soup_i.findAll('li', {'class': 'ingredient'})
#                 ingredient_list.append({'Title': title,
#                                          'ingredient':[normalize_string(ingredient.get_text())for ingredient in ingredients]\
#                                                       if ingredients else np.nan
#                                        }) 
#                 directions = soup_i.findAll('div', class_="instructions")
#                 direction_list.append({'Title': title,
#                                        'Direction': [normalize_string(direction.get_text()) for direction in directions]\
#                                                     if directions else np.nan
#                                       })
#                 print(count, name)
#             else:
#                 null_list.append(name)
#                 print(count, name, 'not recipe pages')
#         else:
#             null_list.append(name)
#             print(count, name, 'null pages')
#         done_list.append(name)
#         count+=1

In [9]:
foodnetwork_dict = []

In [10]:
for root, _, files in walk(FOLDER_PATH):
    for name in files:
        if name in done_list:
            print('The file has been scrapped.')
            continue
        soup_i = BeautifulSoup(open(FOLDER_PATH + name, 'rb'), 'html.parser')
        if soup_i.findAll('a'):    #make sure there is no blank HTML
            if ('http://www.foodnetwork.com/recipes/' in soup_i.text and soup_i.find("meta", property="og:title")): # make sure the page is for recipe
                
                title = soup_i.find("meta", property="og:title")['content']
                
                html = name
                
                if soup_i.find('div',class_="rm-block lead hreview-aggregate review"):
                    rating_tmp = re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)
                else:
                    rating_tmp = []
                    
                if rating_tmp:
                    Rating = float(re.findall(r'Rated (\d+) stars', soup_i.find('div',class_="rm-block lead hreview-aggregate review").text)[0])
                else:
                    Rating = np.nan

                if soup_i.findAll('span', {'class':"lgbtn-text"}):
                    review_tmp = re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)
                else:
                    review_tmp = []
                
                if review_tmp:
                    Review_cnt = int(re.findall('(\d+)',soup_i.findAll('span', {'class':"lgbtn-text"})[1].text)[0])
                else:
                    Review_cnt = np.nan
                
                if soup_i.find('dd',class_="head duration clrfix"):
                    Total_time = get_minutes(soup_i.find('dd',class_="head duration clrfix"))
                else:
                    Total_time = np.nan
                    
                if soup_i.find('dd',class_="cookTime clrfix"):
                    cook_time = get_minutes(soup_i.find('dd',class_="cookTime clrfix"))
                else:
                    cook_time = []
                
                if soup_i.find('dd',class_="prepTime clrfix"):
                    prep_time = get_minutes(soup_i.find('dd',class_="prepTime clrfix"))
                else:
                    prep_time = []
                    
                
                ingredients = soup_i.findAll('li', {'class': 'ingredient'})
                if ingredients:
                    ingredient = [normalize_string(ingredient.get_text())for ingredient in ingredients]
                else:
                    ingredient = np.nan
                    
                directions = soup_i.findAll('div', class_="instructions")
                if directions:
                    direction = [normalize_string(direction.get_text()) for direction in directions]
                else:
                    direction = np.nan

                tmp_dict = {'Title':title,'Rating':Rating,'Review_cnt':Review_cnt,'Total_time':Total_time,\
                            'Prep_time':prep_time,'Cook_time':cook_time,'ingredient':ingredient,\
                            'direction':direction,'html':html}
                
                foodnetwork_dict.append(tmp_dict)
                
                print(count, name)
            else:
                null_list.append(name)
                print(count, name, 'not recipe pages')
        else:
            null_list.append(name)
            print(count, name, 'null pages')
        done_list.append(name)
        count+=1

0 8c7f0e220ed85e905716390ee29c6d3e.html
1 6651dee7da982c5e985aeff2dc761845.html
2 9c5ce84a5203e970e9446dbbbc53c0c1.html
3 86cc555b7751534e5147088a6b481e66.html
4 c6386689e97a75eca597e35474a27f17.html
5 8d29cb18b1beb11855b6aadbedacbc05.html
6 f4c756102b1208dd929269c425b1dfd4.html
7 58b3f783e16c835db13810b8ce00e157.html
8 5ad8db8b0e29a11c3dce271a6b4da748.html
9 bf48fe1dcc02067d55fadb482a1aadeb.html
10 504bfce0a5cdcebbf8c3be357d74cdb1.html
11 2fb65d9f133861bd20abbf21010822b5.html
12 32d43ce80514cbf961cafa1344dc0786.html
13 a0d2a49cf97091cd58e91a8b9cb14c2c.html
14 e5a30ac75ee4e13ccff63e77c8a85b0c.html
15 a1b377af8f59fc29a141eb1f83580df1.html
16 8fa3d2a78081ad60d21cd4a0ee975444.html
17 0941196f6bf9b6d9a191b7b6ea878b3a.html
18 f9f797b13195a2e82bff16d19b459a16.html
19 fae32770f8dfeffa94314c2ba3516f0d.html
20 55ae790ea843d9ecfd790863aa42cd31.html
21 eddf61d270a3a70285bc283306b72b7a.html
22 bb7ea7d75b2fd269142926ce9ee6281b.html
23 d7990522c42e36a4a3cfc59c49ed6f6e.html
24 5e60a566eda4bc75ac74729

198 6799c7f803e2feb849802dec2bc88533.html
199 5f43c349a7a2b25b4e5a18639ffb0abf.html
200 4b40b19dcbe8ec0f9719b822ed031839.html
201 d8ebe57b8e74ae009b93d3c4cb36cb06.html
202 3c432023e63c52ce180dc50a4bab815a.html
203 68f72b2965dec2b21bdaea884e788de9.html
204 ae2e4a44d2d7386a630fa8a0379838e2.html
205 56bdc9668d489e2376fefd87d20986fa.html
206 cb81782340d703ed6b6f755a0c3a4fcc.html
207 09c6e46edd15b7e1072ccdf9bc8efd27.html
208 01295e14041677a3ffa3aa3bc71c8003.html
209 3d917c409b562d62a79ed7c1a2caab2c.html
210 0727b986878291d24a2a7f64c8caca92.html
211 61b883e358c44007d6fe93202b0d8bee.html
212 be9630ba9370e14ac5091cc92cca11af.html
213 9cb5d165b3d4afe56b7d87847ab1f69d.html
214 9658a7921710c1fc13c2fd5cbdf12ed9.html
215 e9a5dbe59d7f07fe594698ff610aca6d.html
216 36fd0464545575254d013d795f35ec4f.html
217 c3cbf2fdc1986ca87829219b12ce5c03.html
218 78c96caea2143af4f63143fa8ec5c0e5.html
219 68d25a4e7307e62172fc1d6cd1458504.html
220 2bee6306f6f680d1c4be61780d6fa56e.html
221 bc6a1835742924d87e6433a6e3b3a0

394 bf84f71a64734301bf26bb16aa550116.html
395 e7a99f9b77b7c56fa0d78c2f026d7749.html
396 f2823a2c71bc617fd800093506be6021.html
397 4c57fa908985b338d3d5f5bcbcf6df20.html
398 25be9fdcd0aff2cd117ff66891ddd2b4.html
399 22edc78affd1244bf6a987aaea0186f6.html
400 bdd96a95b63f12556fabe6e108d9e34d.html
401 7d99e0294f5e71d929f72475c61b4de3.html
402 89ca43c064a81b7843fd1068486ffca9.html
403 a9b69fe66a9983e60ff190155da641e9.html
404 cf86c5f328876fad808e819facd2a1d5.html
405 ae2ee0829c01c8fef7e1eb0871f50d0c.html
406 bc552020ef0ee21f5566d2d1451a5af4.html
407 65497a406599c52d79d0172c63e8e2e3.html
408 49d121ced13858e572e9732f860d53bf.html
409 f603a909c5aae7193fc5d6c8a6e52d7a.html
410 2b25d987edf5ac51dce29354f02bd23d.html
411 f7fa29f57668416fbd0cff8ff38eb846.html
412 2d1f6deb609b93890fc1bb321e634295.html
413 1f56ef6a0b14ce88151ee5d9af7e4966.html
414 e9de43f63bdc9dbb7cb8f0dc4ffd4ea7.html
415 a647bfe1d3faec8816a1df19ec51ad53.html
416 657492ea405cf34a9d64ec03b755b904.html
417 279fcced0d57e41f61fb4484724979

590 e7c0c9619f1e96f251d84b7eda1e5db0.html
591 12afe6121f3e8921cfe1e2833b68fb84.html
592 b10be29ae0bdeafc35fb6129b6faaa9e.html
593 5dd643703c319074f15e925d7a798a4c.html
594 48ee4f2fab4bc84d487b4a157b53d5f3.html
595 57323f2fe0fa478ced0cd6ca1aaff0d0.html
596 d1a75a2c32873beb6c457fcb061b6931.html
597 5d8d7dd5ed63b277058f1bde7d71b540.html
598 2b52dbbfa69026b7b2e128f80532705c.html
599 2c22c09897fadf60b213cd77839e8fde.html
600 f5ee6c23200b1146fca37a3187917dba.html
601 afe004b1abbacfa4dfdf1f4448b96ed4.html
602 c26ca5c8e56df799cd940db267bb0b48.html
603 aae77e510b46e4a45ca6081ee465013e.html
604 92d94237634c5ae4962046d9cff4d06a.html
605 747dbfe8b3fcb384e52f466974b5250b.html
606 1e035c131de84c06af57e5d027afcd3d.html
607 9bcf99f8f95ef9ecba97ed67bf8295e3.html
608 fef9f63a428f096e8fbb921762a02df4.html
609 f4900734346035698d906651e646ed29.html
610 c2bc84abce42ae53e5e3640ee4855c60.html
611 deeb73a8c6b7f8b6d49dede279f6fcd0.html
612 7623cac3050ace8f211b911d5765fd77.html
613 d96b36a79ce7d64d040e1e8f2dcd6b

786 fdb5023de1f5ac9c29c901c5676988f5.html
787 e102a3f6b6199964a0ae9aec96f92c8a.html
788 1f8eda7a83ae9c54271638804ded93f8.html
789 7f63b31a5953197bf5343f5a1690d235.html
790 e2460e133af7ed1e20da4d9bcccce68b.html
791 a02188bd00910ca6fb905b9317b390c9.html
792 2aa4d68c37626bb72f09479a642e8325.html
793 bb858840e0f5e959f6b148007db1cb89.html
794 668958febd589042257d645aa4385982.html
795 53342e85392e7c9cb0d8224609c5babf.html
796 cbb3dd785fde7fd15888c6031893b602.html
797 29fb820e19621fb485b1beeb0736928f.html
798 9bbcd6efcc3e8ea1e854ddd472048fdc.html
799 4d0e8887f15e471aad492bfb9dc4f360.html
800 aca625506fe1b396d18edf471ccadf7f.html
801 ebf43c0bdb69bcb85726b47c0cb4eed0.html
802 a78d9baa665065a7be9be58a6804bc6f.html
803 f92ef99dfe1cbab5be31c2d80abfa02b.html
804 703e163139ce5ff8617905ccf82c56cc.html
805 d52e416434e0b8e624e1254f2b7c5857.html
806 882e91c18265461ec404563082eee8ba.html
807 ffda723cdafd101359affce44e8fdc64.html
808 53cf713adfaf5fa450185333c04f0912.html
809 f5bc6e8930af3143877ef40aa4fa4c

...

6714 4dd3ad01527b61b89af378c78011bb5c.html
6715 dfd5a9e09944ec6287dabf7fac8df437.html
6716 00803850c2d4597052f48f0e9875baf1.html
6717 5db901f79446500fba5a39744b52097b.html
6718 4721c2960e7790a6d2a7093c0d0fa7fa.html
6719 f75853e74a81d5d847f8d1e29d4d7868.html
6720 4950bf398916af7c4f0d27f5845b2060.html
6721 d0740e767606820cf11bcf860b78536f.html
6722 cdcb39789c31e6a532bd9b2852065bc4.html
6723 f7fe6a0e4b8442bfe946e4d8f248647f.html
6724 27898b1e2d9ad50ba5fbf5b1bd918548.html
6725 bef5c869c34f0294437d394b889b11af.html
6726 ea99a13efa27df2bb9cee7dc77ffe36a.html
6727 a61621f0825c11a09fe4848bb1ee67cf.html
6728 a1760063dc58dd4115236d38b16c502d.html
6729 e9fe9a8c65004de90f23e50fe4f05f33.html
6730 8f4e05c1defd6dae82d6bd4991b1f559.html
6731 107c4a657372433ec7a23e6dd98732f1.html
6732 a94d130fa946d869c9c6797f77da187f.html
6733 6912a2baa51aee4e277c3fe06818584c.html
6734 b8ae750e48b4f983e59298d794671822.html
6735 ee96bc446fe9b64836ee0a788b36f3d4.html
6736 bdc8ad057428cd65b2d2607569802b9d.html
6737 f5cbbe

6905 7883a8eeb1564e4fbcabac063a5d625e.html
6906 9cee9a6f9d89eb5887c9b3dbd213a979.html
6907 2c93fc6f604fd030515c3576fca2bfd9.html
6908 9ef6bb2cf167af93472b69aa7fc9162b.html
6909 8cf2869001e5ebc029dab04940bc708b.html
6910 998d415eecc681855858e15dddbfd5b7.html
6911 ac5aeb50183bcde59ae16341f8c5436b.html
6912 3cc45716535ff04533422506bd0399e6.html
6913 bb6c5be8d28e544f482b4d650214e06a.html
6914 13cf9d7af1445918c878cca924986c9b.html
6915 9f5679def7e6b6bb8ada9ce59b5b3252.html
6916 55f6fe1cb2e1c25af34b3e4e647854b9.html
6917 9df5ec4899853eba99abf1ea9f71db6a.html
6918 f74d8b6349101df22eb01604d73310c0.html
6919 33199eab7a7e1275ede05495a5fe5eb3.html
6920 43e59fa39cce1b52539b0b8dada3fb4f.html
6921 8b03e243d2c11a084988b78e69348bb8.html
6922 19c50373bf5e7b11592f1565608a14dc.html
6923 09608693dd81c2ff801e864d4ab41fc0.html
6924 22a4bb44e044024ff48b5cde5c03c602.html
6925 d7f71ed238ca4c49077c00b8e5b1b204.html
6926 28841bb7c93143741e11b4699dbcbb74.html
6927 5316613b8658ae0ba3d2ff840b6423dd.html
6928 ad17fb

7096 17aa557c70ad46aead97ef8e165afbc1.html
7097 274d040fc6f9cf0b073c3d17c2c184b9.html
7098 76765b48a26aa0ef4576b7e9abfc0a74.html
7099 5e058d91471a37007db1a6bec41c68f8.html
7100 c6419e1de197651bf3eea1f4b46c70cb.html
7101 6dbf000654a6a476dfc449375ecbdbfe.html
7102 fa19636808b3760d29eecc90b8f29c02.html
7103 dc8f16ca4edecbc1684840f95ec4e98e.html
7104 367f16e19ba4c83d285e52c216e642e6.html
7105 0b1c6090a65b97d867572b16393e5123.html
7106 f7bdff652866c1a3d06af68b14ddb8a6.html
7107 48aa8d3772dccb7b4c6a18abea81d920.html
7108 c4998fea28a554cca04e5a65f5481430.html
7109 10e6b900b1b2e19bbe3fc4496a625c4a.html
7110 0769f0b5067561a9f4f1839068957424.html
7111 068c5738ce51eb8259c59f40924b5a2f.html
7112 b6f6364173e951291527c40eb33e2b4f.html
7113 20b765436dbb0ec0cae3397a48268b1d.html
7114 b1a707978a3e072cc761a035949b8260.html
7115 03a25280e3d43d7b8f3e02b1cd62628f.html
7116 25dd0fa14d231c20ec41c5352fdcb331.html
7117 a789136b266ef104fb867abe49a6d141.html
7118 19092f6900b98ea13670f171998b2a62.html
7119 548fa0

7287 c8f741536ce0ff3bd6b710256bdcf0da.html
7288 e56adf627d957d25b90f3bf5f886c811.html
7289 5e38537e9283e079bfae42718b912130.html
7290 e615ff205572a807296a108d38fc5c5e.html
7291 f99d5f4878b51221fae174b82112d625.html
7292 cee3d32e22bc82afbea3a9f170b0dcac.html
7293 5be86a71818a79da5edfb833abfd5571.html
7294 77847774d1d9c7338a17a01cc8062120.html
7295 01c679a8d9374c20f89d1cb010fbdd17.html
7296 37cb51324bad54695b467102bb094e0d.html
7297 8fa1b3b3bd0528937f1e0c2329da08b6.html
7298 b7347c54a730ac06abe1f01689349dca.html
7299 05c0ec91b20a8cab78824b460cb08a7e.html
7300 be28db61887738957bcff91624453d69.html
7301 47ec0455e8815b54217bd908566fcc13.html
7302 7d3230523c8fa665ad9f72c1c4c8c2b4.html
7303 29dde927a306a20ab5e0d71467b821b9.html
7304 572ac237c95552c68e70b6ea870bc163.html
7305 ad7a7d6f1fd4b4c6edb09f07532f72ef.html
7306 ec1341adb7e5e310c6bf2768bb2939b6.html
7307 e0097a7a5770945c1b84ffb0e497fcc2.html
7308 7886d8193b692ce6116be10813bb300f.html
7309 c5b4ea19d25acadc4c5c7ca99c2628b7.html
7310 70d155

7478 49c637996eb9c7ff52826ccff1a531d2.html
7479 70d9e239b6ceb3d7a01de7a12ff27158.html
7480 3811bcdc504d6dec207eb9983a2ee196.html
7481 d10a3358e63b893fe725378b916d8a49.html
7482 aefb14e34bd20801f3793a186e3b040e.html
7483 960c558e6e5578d0f7b02f0b64a5d0d8.html
7484 ed4dc7857fd0cd186f688348b44f3fed.html
7485 c86bdab5339a3b5c62122fe8578a8baf.html
7486 d59bf1e0798d2cc11958b81e471d290f.html
7487 0e3deb7758a96f48d2ed1f90277be4be.html
7488 b8bbbeff69cbe041b3ebb0678ba6871b.html
7489 1f258933ac5c5c5d99d2039395dc086f.html
7490 6a6d5ecbba7cd382f705420c861b652b.html
7491 7e6ff85ca8c4fc54bc88c7707ae529f6.html
7492 ad38e43aefd2870780b87c7f9a17500f.html
7493 470a4c9df78682c7b2f1b31da1f91f89.html
7494 4665d428dfd73d5f3a0ba4c2d3d4c3fd.html
7495 d458dc4616b9d056361ae20a04feaf9c.html
7496 71d8a465d802b8cd1d833a6e3001ed18.html
7497 a7ddd4087a5f1a859fff3daec24a6043.html
7498 a53c2246d4b0909be881a5050b57944a.html
7499 2fff1139705d42a523d4275d626c060e.html
7500 d09903ec50e20e453f89ce807343bfd9.html
7501 18b95a

In [12]:
df_foodnetwork = pd.DataFrame(foodnetwork_dict)

In [14]:
df_foodnetwork.drop_duplicates('Title').count()

Cook_time     6685
Prep_time     6685
Rating        6527
Review_cnt    6682
Title         6685
Total_time    6682
direction     6669
html          6685
ingredient    6588
dtype: int64

In [15]:
df_foodnetwork[df_foodnetwork.duplicated(subset=['Title'], keep='first')]

,Cook_time,Prep_time,Rating,Review_cnt,Title,Total_time,direction,html,ingredient
168,[],10,4.0,1.0,Cucumber Salad,10.0,[Simply toss together.],a348c93117f0e0d4d88b2fa9cfdb63ba.html,"[2 hothouse cucumbers, peeled and sliced, 1 sm..."
299,45,30,4.0,6.0,Pecan Pie,75.0,"[In a large bowl, place the lard and butter. A...",eaf07104b8527ff8d515c7c4fdbf3e5e.html,"[1 cup lard, 2 tablespoons butter, 2 cups flou..."
449,15,5,3.0,2.0,Guacamole,20.0,"[Place the chile, onion, and cilantro into the...",7f9660af26ddbb52d5b436fa041fc329.html,"[1 serrano chile, seeds removed and minced, 1/..."
480,[],[],4.0,106.0,Coconut Macaroons,0.0,[Preheat the oven to 325 degrees F. Combine th...,a1036de3f3e663acf05270cbd4c938dc.html,"[14 ounces sweetened shredded coconut, 14 ounc..."
540,75,20,5.0,100.0,Pecan Pie,215.0,"[Make the dough by hand: In a medium bowl, whi...",c88840199dbade7ef099c484eddee30e.html,"[1 1/4 cups all-purpose flour, 2 teaspoons sug..."
549,40,10,5.0,52.0,Baked Ziti,50.0,[Preheat the oven to 400 degrees F. Bring a la...,496d1a46cacfabdbcc82770fde2631a8.html,"[1 pound dried ziti pasta, Kosher salt, 3 1/2 ..."
580,[],[],5.0,389.0,Chocolate Peanut-Butter No Bake Cookies,0.0,"[In a heavy saucepan bring to a boil, the suga...",08abf182f95bd8831a663a8fc1fd3b9d.html,"[2 cups sugar, 4 tablespoons cocoa, 1 stick bu..."
591,30,15,5.0,62.0,Marinara Sauce,45.0,[Heat the olive oil in a large (12-inch) skill...,12afe6121f3e8921cfe1e2833b68fb84.html,"[1 tablespoon good olive oil, 1 cup chopped ye..."
602,20,5,4.0,25.0,Chocolate Mousse,25.0,[Break chocolate into smaller pieces and melt ...,c26ca5c8e56df799cd940db267bb0b48.html,"[6 (4-ounce) bars bittersweet chocolate, 2/3 c..."
656,[],10,4.0,82.0,Cucumber Salad,30.0,"[Slice the cucumber lengthwise, remove the see...",8e71542bb9f9fc224cf225b43f1dcd8f.html,"[2 English cucumbers (2 pounds), 1 small red o..."


In [16]:
df_foodnetwork.loc[df_foodnetwork.Title=='Cucumber Salad']

,Cook_time,Prep_time,Rating,Review_cnt,Title,Total_time,direction,html,ingredient
5,[],10,4.0,1.0,Cucumber Salad,10.0,[Simply toss together.],8d29cb18b1beb11855b6aadbedacbc05.html,"[2 hothouse cucumbers, peeled and sliced, 1 sm..."
168,[],10,4.0,1.0,Cucumber Salad,10.0,[Simply toss together.],a348c93117f0e0d4d88b2fa9cfdb63ba.html,"[2 hothouse cucumbers, peeled and sliced, 1 sm..."
656,[],10,4.0,82.0,Cucumber Salad,30.0,"[Slice the cucumber lengthwise, remove the see...",8e71542bb9f9fc224cf225b43f1dcd8f.html,"[2 English cucumbers (2 pounds), 1 small red o..."
1777,[],10,4.0,1.0,Cucumber Salad,10.0,[Simply toss together.],eb19e2953507fc20b633732c223789fe.html,"[2 hothouse cucumbers, peeled and sliced, 1 sm..."
4071,[],10,4.0,82.0,Cucumber Salad,30.0,"[Slice the cucumber lengthwise, remove the see...",fe3309ace1c281076a52c6965ee69bb5.html,"[2 English cucumbers (2 pounds), 1 small red o..."


In [96]:
with open('./SavedList/foodnetwork_not_recipe.txt', 'w') as f:
    for item in null_list:
        f.write("%s\n" % item)

In [164]:
PATH = '/Users/yan/Documents/GitHub/ADA/Project/dataset/'

PATHSRC = PATH+'foodnetwork/'

PATHDES = PATH+'nullhtml/'

# move html from all recipePages to corresponding folder
for name in null_list:
    src = PATHSRC+name
    #print(src)
    des = PATHDES
    shutil.move(src,des)

In [111]:
df_test = pd.DataFrame(rating_review_list)

In [112]:
df_test.drop_duplicates().count()

Rating     7140
Reviews    7298
Title      7301
dtype: int64

In [115]:
pd.DataFrame(rating_review_list).count()

Rating     7415
Reviews    7575
Title      7578
dtype: int64

In [117]:
df = pd.DataFrame(rating_review_list).merge(pd.DataFrame(times_list))\
.merge(pd.DataFrame(ingredient_list)).merge(pd.DataFrame(direction_list))
cols = list(df)
cols.insert(0, cols.pop(cols.index('Title')))
df = df.loc[:, cols]
df.drop_duplicates(subset=['Title'], inplace=True)
df.reset_index(inplace=True)
df.drop(columns='index')

,Title,Rating,Reviews,Cook Time (min),Prep Time (min),Total Time (min),ingredient,Direction
0,French Onion Dip,5.0,52.0,40.0,20.0,150.0,"[2 tablespoons canola oil, 1 tablespoon butter...","[In a large saute pan over medium high heat, a..."
1,Swedish Meatballs,5.0,271.0,25.0,30.0,55.0,"[2 slices fresh white bread, 1/4 cup milk, 3 t...",[Preheat oven to 200 degrees F. Tear the bread...
2,Sauteed Yellow Squash,5.0,35.0,14.0,10.0,24.0,"[1 tablespoon extra-virgin olive oil, 2 tables...",[Preheat skillet over medium high heat. Add oi...
3,Butternut Squash Lasagna,5.0,385.0,85.0,30.0,130.0,"[1 tablespoon olive oil, 1 (1 1/2 to 2-pound) ...",[Heat the oil in a heavy large skillet over me...
4,Emeril's Essence,5.0,74.0,NaN,NaN,0.0,"[2 1/2 tablespoons paprika, 2 tablespoons salt...",[Combine all ingredients thoroughly.]
5,Cucumber Salad,4.0,1.0,NaN,10.0,10.0,"[2 hothouse cucumbers, peeled and sliced, 1 sm...",[Simply toss together.]
6,Scallops Provencal,5.0,337.0,10.0,5.0,15.0,"[1 pound fresh bay or sea scallops, Kosher sal...","[If you're using bay scallops, keep them whole..."
7,Baked Cheese Grits,5.0,13.0,60.0,5.0,70.0,"[1/4 cup plus 2 tablespoons butter, 4 cups wat...",[Preheat oven to 350 degrees F. Grease a 2-qua...
8,American Macaroni Salad,5.0,223.0,10.0,20.0,30.0,"[2 cups dry elbow macaroni, cooked, rinsed, an...","[In a large bowl combine the macaroni, celery,..."
9,Heirloom Carrot Spoon Cake,4.0,32.0,270.0,3.0,273.0,[1 (18.25-ounce) box spice cake mix (recommend...,[Spray a 5-quart slow cooker with butter flavo...


In [134]:
foodnetwork_dict = []
for i in range(len(rating_review_list)):
    title = rating_review_list[i]['Title']
    Rating = rating_review_list[i]['Rating']
    Review_cnt = rating_review_list[i]['Reviews']
    Total_time = times_list[i]['Total Time (min)']
    Prep_time = times_list[i]['Prep Time (min)']
    Cook_time = times_list[i]['Cook Time (min)']
    ingredient = ingredient_list[i]['ingredient']
    direction = direction_list[i]['Direction']
    tmp_dict = {'Title':title,'Rating':Rating,'Review_cnt':Review_cnt,'Total_time':Total_time,
               'Prep_time':Prep_time,'Cook_time':Cook_time,'ingredient':ingredient,'direction':direction}
    foodnetwork_dict.append(tmp_dict)

In [142]:
df_foodnetwork = pd.DataFrame(foodnetwork_dict)

In [151]:
df_foodnetwork[df_foodnetwork.duplicated(subset=['Title'], keep='first')]

,Cook_time,Prep_time,Rating,Review_cnt,Title,Total_time,direction,ingredient
168,NaN,10.0,4.0,1.0,Cucumber Salad,10.0,[Simply toss together.],"[2 hothouse cucumbers, peeled and sliced, 1 sm..."
299,45.0,30.0,4.0,6.0,Pecan Pie,75.0,"[In a large bowl, place the lard and butter. A...","[1 cup lard, 2 tablespoons butter, 2 cups flou..."
449,15.0,5.0,3.0,2.0,Guacamole,20.0,"[Place the chile, onion, and cilantro into the...","[1 serrano chile, seeds removed and minced, 1/..."
480,NaN,NaN,4.0,106.0,Coconut Macaroons,0.0,[Preheat the oven to 325 degrees F. Combine th...,"[14 ounces sweetened shredded coconut, 14 ounc..."
540,75.0,20.0,5.0,100.0,Pecan Pie,215.0,"[Make the dough by hand: In a medium bowl, whi...","[1 1/4 cups all-purpose flour, 2 teaspoons sug..."
549,40.0,10.0,5.0,52.0,Baked Ziti,50.0,[Preheat the oven to 400 degrees F. Bring a la...,"[1 pound dried ziti pasta, Kosher salt, 3 1/2 ..."
580,NaN,NaN,5.0,389.0,Chocolate Peanut-Butter No Bake Cookies,0.0,"[In a heavy saucepan bring to a boil, the suga...","[2 cups sugar, 4 tablespoons cocoa, 1 stick bu..."
591,30.0,15.0,5.0,62.0,Marinara Sauce,45.0,[Heat the olive oil in a large (12-inch) skill...,"[1 tablespoon good olive oil, 1 cup chopped ye..."
602,20.0,5.0,4.0,25.0,Chocolate Mousse,25.0,[Break chocolate into smaller pieces and melt ...,"[6 (4-ounce) bars bittersweet chocolate, 2/3 c..."
656,NaN,10.0,4.0,82.0,Cucumber Salad,30.0,"[Slice the cucumber lengthwise, remove the see...","[2 English cucumbers (2 pounds), 1 small red o..."


In [156]:
df_foodnetwork.loc[df_foodnetwork.Title=='Creamy Cole Slaw']

,Cook_time,Prep_time,Rating,Review_cnt,Title,Total_time,direction,ingredient
218,NaN,10.0,4.0,214.0,Creamy Cole Slaw,10.0,[Combine the shredded cabbage and carrots in a...,"[1 head green cabbage, finely shredded, 2 larg..."
4056,NaN,10.0,4.0,214.0,Creamy Cole Slaw,10.0,[Combine the shredded cabbage and carrots in a...,"[1 head green cabbage, finely shredded, 2 larg..."
7174,NaN,10.0,4.0,213.0,Creamy Cole Slaw,10.0,[Combine the shredded cabbage and carrots in a...,"[1 head green cabbage, finely shredded, 2 larg..."


In [17]:
df_foodnetwork.to_csv('./SavedList/foodnetwork_fined.csv')